In [1]:
import os
import pandas as pd
import uuid
import json
import vertexai
from vertexai import agent_engines
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Callable, Dict, Optional, Union, Any
# Importy z własnych modułów
from config import PROJECT_ID, LOCATION, MEMORY_ENGINE_DISPLAY_NAME, INPUT_FILE_PATH,MAIN_AGENT,CRITIC_MODEL,CODE_MODEL, API_TYPE_GEMINI,API_TYPE_SONNET, ANTHROPIC_API_KEY,basic_config_agent
from agents.state import AgentWorkflowState
from agents.autogen_agents import TriggerAgent,PlannerAgent,CriticAgent
from prompts import LangchainAgentsPrompts,AutoGenAgentsPrompts
from prompts_beta import PromptFactory
from agents.langgraph_nodes import * 
from agents.autogen_agent_utils import run_autogen_planning_phase
from memory.memory_bank_client import MemoryBankClient
from tools.utils import *

In [2]:
AGENT_ENGINE_NAME = "" # Zostanie wypełniona po pobraniu lub utworzeniu silnika

# Inicjalizacja głównego klienta Vertex AI
client = vertexai.Client(project=PROJECT_ID, location=LOCATION)

In [3]:
def get_or_create_agent_engine(display_name: str) :
    """
    Pobiera istniejący Agent Engine po nazwie wyświetlanej lub tworzy nowy, jeśli nie istnieje.
    """
    # 1. Pobierz listę wszystkich istniejących silników w projekcie
    all_engines = agent_engines.list()
    
    # 2. Sprawdź, czy któryś z nich ma pasującą nazwę
    for engine in all_engines:
        if engine.display_name == display_name:
            print(f"INFO: Znaleziono i połączono z istniejącym Agent Engine: '{display_name}'")
            return engine
            
    # 3. Jeśli pętla się zakończyła i nic nie znaleziono, stwórz nowy silnik
    print(f"INFO: Nie znaleziono Agent Engine o nazwie '{display_name}'. Tworzenie nowego...")
    try:
        new_engine = agent_engines.create(
            display_name=display_name
        )
        print(f"INFO: Pomyślnie utworzono nowy Agent Engine.")
        return new_engine
    except Exception as e:
        print(f"KRYTYCZNY BŁĄD: Nie można utworzyć Agent Engine. Sprawdź konfigurację i uprawnienia. Błąd: {e}")
        exit()


In [4]:
agent_engine =get_or_create_agent_engine(MEMORY_ENGINE_DISPLAY_NAME)
AGENT_ENGINE_NAME = agent_engine.resource_name
print(AGENT_ENGINE_NAME)


INFO: Znaleziono i połączono z istniejącym Agent Engine: 'memory-gamma-way'
projects/815755318672/locations/us-central1/reasoningEngines/3849548538518175744


In [5]:
# --- Konfiguracja czatu grupowego ---
main_agent_configuration={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(agent_name=MAIN_AGENT, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}
critic_agent_configuration ={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(api_key=ANTHROPIC_API_KEY,agent_name=CRITIC_MODEL, api_type=API_TYPE_SONNET)}

#---WYWOŁANIE AGENTÓW
trigger_agent = TriggerAgent(llm_config=main_agent_configuration, prompt=PromptFactory.for_trigger())
planner_agent = PlannerAgent(llm_config=main_agent_configuration, prompt=PromptFactory.for_planner()) # Tutaj nie przekazujemy inspiracji
critic_agent = CriticAgent(llm_config=critic_agent_configuration, prompt=PromptFactory.for_critic())

In [6]:
if __name__ == "__main__":
    
    files_to_exclude = {'Agents_beta (10).py','pack_project.ipynb', 'caly_projekt.txt'}
    system_source_code = read_project_source_code(".", exclude_files=files_to_exclude)

    # --- Inicjalizacja Pamięci i Uruchomienia ---
    memory_client = MemoryBankClient(client=client, agent_engine=agent_engine)
    run_id = str(uuid.uuid4())
    
    print("\n--- ODPYTYWANIE PAMIĘCI O INSPIRACJE ---")
    inspiration_prompt = ""
    dataset_signature = ""
    try:
        df_preview = pd.read_csv(INPUT_FILE_PATH, nrows=0)
        dataset_signature = memory_client.create_dataset_signature(df_preview)
        past_memories = memory_client.query_memory(
            query_text="Najlepsze strategie i kluczowe wnioski dotyczące przetwarzania danych",
            scope={"dataset_signature": dataset_signature},
            top_k=3
        )
        if past_memories:
            inspirations = []
            for mem in past_memories:
                # ZMIANA: Używamy nowych, poprawnych typów wspomnień
                if mem.memory_type == MemoryType.SUCCESSFUL_WORKFLOW and 'key_planning_insight' in mem.content:
                    inspirations.append(f"SPRAWDZONY WNIOSEK Z PROCESU: {mem.content['key_planning_insight']}")
                elif mem.memory_type == MemoryType.SUCCESSFUL_FIX and 'key_takeaway' in mem.content:
                    inspirations.append(f"NAUCZKA Z NAPRAWIONEGO BŁĘDU: {mem.content['key_takeaway']}")
            if inspirations:
                inspiration_prompt = "--- INSPIRACJE Z POPRZEDNICH URUCHOMIEŃ ---\n" + "\n".join(inspirations)
                print("INFO: Pomyślnie pobrano inspiracje z pamięci.")
        else:
            print("INFO: Nie znaleziono inspiracji w pamięci dla tego typu danych.")
    except Exception as e:
        print(f"OSTRZEŻENIE: Nie udało się pobrać inspiracji z pamięci: {e}")

        
        
    active_policies = get_active_policies_from_memory(memory_client, dataset_signature)    
    
    # --- Krok 1: Faza planowania (AutoGen) ---
    final_plan, autogen_log = run_autogen_planning_phase(
        input_path=INPUT_FILE_PATH, 
        inspiration_prompt=inspiration_prompt,
        trigger_agent=trigger_agent,
        planner_agent=planner_agent,
        critic_agent=critic_agent,
        manager_agent_config=main_agent_configuration,
        active_policies=active_policies
    )
    save_autogen_conversation_log(log_content=autogen_log, file_path="reports/autogen_planning_conversation.log")

    # --- Krok 2: Faza wykonania (LangGraph) ---
    if final_plan:
        print("\n" + "="*80)
        print("### ### FAZA 2: URUCHAMIANIE WYKONANIA PLANU (LangGraph) ### ###")
        print("="*80 + "\n")
        
        workflow = StateGraph(AgentWorkflowState)
        
        # <<< ZMIANA TUTAJ: Zaktualizowana lista węzłów >>>
        nodes = [
            "schema_reader", "code_generator", "architectural_validator", 
            "data_code_executor", "universal_debugger", "apply_code_fix", 
            "human_approval", "package_installer", "human_escalation", 
            "sync_report_code","meta_auditor",
            # Nowe, wyspecjalizowane węzły raportujące:
            "summary_analyst", "plot_generator", "report_composer","memory_consolidation"
        ]
        for name in nodes: workflow.add_node(name, globals()[f"{name}_node"])

        # Definicja krawędzi
        workflow.set_entry_point("schema_reader")
        
        # Ścieżka przetwarzania danych
        workflow.add_edge("schema_reader", "code_generator")
        workflow.add_edge("code_generator", "architectural_validator")
        
        def should_continue_or_debug(state: AgentWorkflowState) -> str:
            if state.get("error_message"):
                if state.get("correction_attempts", 0) >= MAX_CORRECTION_ATTEMPTS:
                    return "request_human_help"
                return "call_debugger"
            return "continue"

        workflow.add_conditional_edges(
            "architectural_validator",
            should_continue_or_debug,
            {"call_debugger": "universal_debugger", "request_human_help": "human_escalation", "continue": "data_code_executor"}
        )
        workflow.add_conditional_edges(
            "data_code_executor",
            should_continue_or_debug,
            {"call_debugger": "universal_debugger", "request_human_help": "human_escalation", "continue": "summary_analyst"}
        )
        
        workflow.add_conditional_edges(
            "summary_analyst",
            should_continue_or_debug,
            {"call_debugger": "universal_debugger", "request_human_help": "human_escalation", "continue": "plot_generator"}
        )
        workflow.add_conditional_edges(
            "plot_generator",
            should_continue_or_debug,
            {"call_debugger": "universal_debugger", "request_human_help": "human_escalation", "continue": "report_composer"}
        )
        workflow.add_conditional_edges(
            "report_composer",
            should_continue_or_debug,
            {"universal_debugger": "universal_debugger", "human_escalation": "human_escalation", "continue": "meta_auditor"}
        )

        # Ścieżki naprawcze
        workflow.add_edge("human_escalation", "meta_auditor")
        workflow.add_edge("package_installer", "data_code_executor")

        def route_after_fix(state):
            failing_node = state.get("failing_node")
            # Po naprawie wraca do węzła, który zawiódł
            if failing_node:
                return failing_node
            # Domyślnie wraca do walidacji
            return "architectural_validator"

        workflow.add_conditional_edges("apply_code_fix", route_after_fix)

        def route_from_debugger(state):
            if state.get("tool_choice") == "propose_code_fix":
                return "apply_code_fix"
            if state.get("tool_choice") == "request_package_installation":
                return "human_approval"
            return "human_escalation"

        workflow.add_conditional_edges("universal_debugger", route_from_debugger)
        workflow.add_conditional_edges("human_approval", lambda s: s.get("user_approval_status"), {
            "APPROVED": "package_installer", "REJECTED": "universal_debugger"
        })

        
        workflow.add_edge("human_escalation", "meta_auditor")
        workflow.add_edge("meta_auditor", "memory_consolidation")
        workflow.add_edge("memory_consolidation", END)
        
        app = workflow.compile()
        
        app_config = {"MAIN_AGENT": MAIN_AGENT, "CODE_MODEL": CODE_MODEL, "CRITIC_MODEL": CRITIC_MODEL}
        
        initial_state = {
            "config": app_config,
            "plan": final_plan, 
            "input_path": INPUT_FILE_PATH,
            "output_path": "reports/processed_data.csv",
            "report_output_path": "reports/transformation_report.html",
            "correction_attempts": 0, 
            "correction_history": [],
            "source_code": system_source_code,
            "autogen_log": autogen_log,
            "memory_client": memory_client,
            "run_id": run_id,
            "dataset_signature": dataset_signature,
            "pending_fix_session": None,
            "active_policies": active_policies
        }
        
        langgraph_log = ""
        final_run_state = initial_state.copy()
        
        for event in app.stream(initial_state, {"recursion_limit": 50}):
            for node_name, state_update in event.items():
                if "__end__" not in node_name:
                    print(f"--- Krok: '{node_name}' ---")
                    if state_update:
                        printable_update = state_update.copy()
                        for key in ["generated_code", "corrected_code", "generated_report_code", "error_context_code", "plot_generation_code", "summary_html"]:
                            if key in printable_update and printable_update[key]:
                                print(f"--- {key.upper()} ---")
                                print(printable_update[key])
                                print("-" * (len(key) + 8))
                                del printable_update[key]
                        if printable_update:
                            print(json.dumps(printable_update, indent=2, default=str))
                        
                        log_line = f"--- Krok: '{node_name}' ---\n{json.dumps(state_update, indent=2, default=str)}\n"
                        langgraph_log += log_line
                        final_run_state.update(state_update)
                    else:
                        print("  [INFO] Węzeł zakończył pracę bez aktualizacji stanu.")
                    print("-" * 20 + "\n")

        save_langgraph_execution_log(log_content=langgraph_log, file_path="reports/langgraph_execution.log")

        final_run_state['langgraph_log'] = langgraph_log
        meta_auditor_node(final_run_state)

        print("\n\n--- ZAKOŃCZONO PRACĘ GRAFU I AUDYT ---")
    else:
        print("Proces zakończony. Brak planu do wykonania.")

INFO: MemoryBankClient gotowy do pracy z silnikiem: projects/815755318672/locations/us-central1/reasoningEngines/3849548538518175744

--- ODPYTYWANIE PAMIĘCI O INSPIRACJE ---
INFO: Odpytuję pamięć semantycznie z zapytaniem 'Najlepsze strategie i kluczowe wnioski dotyczące przetwarzania danych' w zakresie {'dataset_signature': 'ae1568fe7dae11d4bacd0c21ed718503'}


/home/jupyter/olga_zydziak/version_beta/multiagent_system/memory/memory_bank_client.py:95: ExperimentalWarning: The Vertex SDK GenAI agent engines module is experimental, and may change in future versions.
  memories_iterator = self.client.agent_engines.retrieve_memories(


udany plan: id='b91d3a82-dd82-47ff-a12b-d0884e12d495' run_id='d1a03c21-e9a8-42eb-88e0-74d8e94ed71f' timestamp=datetime.datetime(2025, 8, 6, 23, 6, 37, 684735) memory_type=<MemoryType.SUCCESSFUL_WORKFLOW: 'SUCCESSFUL_WORKFLOW'> dataset_signature='ae1568fe7dae11d4bacd0c21ed718503' source_node='memory_consolidation_node' content={'workflow_summary': 'Pomyślnie przetworzono zbiór danych, stosując zaawansowane techniki czyszczenia, kategoryzacji i skalowania w celu przygotowania go do modelowania.', 'key_planning_insight': 'Kluczowym wnioskiem z fazy planowania było zróżnicowanie strategii skalowania cech: zaplanowano użycie `RobustScaler` dla kolumn z istotnymi wartościami odstającymi (`Transaction_Amount`, `Account_Balance`) oraz `StandardScaler` dla pozostałych cech numerycznych. Takie podejście proaktywnie adresuje problem outlierów.', 'key_execution_insight': 'Najważniejszą obserwacją z fazy wykonania była skuteczna implementacja podwójnej strategii skalowania. Kod poprawnie zidentyfik

/opt/conda/envs/agents_with_memory_py11/lib/python3.11/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


TriggerAgent (to chat_manager):

Dane są tabularyczne. Przekazuję do PlannerAgent w celu stworzenia planu analizy.

--------------------------------------------------------------------------------

Next speaker: PlannerAgent


>>>>>>>> USING AUTO REPLY...
PlannerAgent (to chat_manager):

**CEL:** Stworzenie planu przygotowania danych do modelowania predykcyjnego w celu wykrywania oszustw.
**ANALIZA:** Dane wejściowe zawierają mieszankę typów danych: numeryczne, kategoryczne, binarne oraz znacznik czasu. Wymagane jest czyszczenie, inżynieria cech i skalowanie w celu przygotowania spójnego zbioru danych dla algorytmów uczenia maszynowego. Inspiracja dotycząca zróżnicowanego skalowania jest trafna i zostanie zaadaptowana.
**WNIOSEK:** Poniższy plan przedstawia sekwencję kroków niezbędnych do transformacji surowych danych w gotowy do analizy zbiór cech.

---
### **PLAN PRZYGOTOWANIA DANYCH [cite: 41, 56]**

**Faza 1: Wstępna inspekcja i walidacja**

1.  **Wczytanie danych:** Załaduj zbiór 

  [AUDYT] ⚠️ OSTRZEŻENIE: Nie udało się wygenerować wniosku META z raportu audytora. Błąd: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'This request would exceed the rate limit for your organization (43528422-5f54-43d4-a170-815c85dadcea) of 20,000 input tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}
--- Krok: 'meta_auditor' ---
  [INFO] Węzeł zakończył pracę bez aktualizacji stanu.
--------------------


### ### FAZA 4: KONSOLIDACJA WIEDZY W PAMIĘCI ### ###

  [PAMIĘĆ] Wykryto pomyślny przebieg bez błędów. Zapisuję wspomnienie typu SUCCESSFUL_WORKFLOW.
  [PAMIĘĆ] Uruchamiam destylację całego udanego procesu...
  [PAMIĘĆ] ✅ Pomyślnie przedestylowano